In [ ]:
import json
import os
import re
from collections import namedtuple
from urllib.parse import urlparse
import imagehash
import mercantile
from shapely.geometry import shape, Point
from PIL import Image
from io import BytesIO
from collections import defaultdict
import matplotlib.pyplot as plt
from pyproj import Transformer
from pyproj.crs import CRS
import requests
from ipywidgets import interact, interactive, Text, interact_manual, HTML

style = """
    <style>
        .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
    </style>
    """
display(HTML(style))

In [ ]:
def max_count(elements):
    counts = defaultdict(int)
    for el in elements:
        counts[el] += 1
    return max(counts.items(), key=lambda x: x[1])[1]


def get_url(url, session, with_text=False, with_data=False, headers=None):
    r = session.get(url, headers=headers)
    print("Request: {} -> {}".format(url, r.status_code))
    if not r.status_code == 200:
        return None
    if with_text:
        return r.text
    if with_data:
        return r.content

    
def get_http_headers(source):
    """ Extract http headers from source"""
    headers = {}
    if 'custom-http-headers' in source['properties']:
        key = source['properties']['custom-http-headers']['header-name']
        value = source['properties']['custom-http-headers']['header-value']
        headers[key] = value
    return headers


def get_tms_image(tile, source, session):
    tms_url = source['properties']['url']
    parameters = {}
    # {z} instead of {zoom}
    if '{z}' in tms_url:
        return
    if '{apikey}' in tms_url:
        return

    if "{switch:" in tms_url:
        match = re.search(r'switch:?([^}]*)', tms_url)
        switches = match.group(1).split(',')
        tms_url = tms_url.replace(match.group(0), 'switch')
        parameters['switch'] = switches[0]

    extra_headers = get_http_headers(source)
    query_url = tms_url
    if '{-y}' in tms_url:
        y = 2 ** tile.z - 1 - tile.y
        query_url = query_url.replace('{-y}', str(y))
    elif '{!y}' in tms_url:
        y = 2 ** (tile.z - 1) - 1 - tile.y
        query_url = query_url.replace('{!y}', str(y))
    else:
        query_url = query_url.replace('{y}', str(tile.y))
    parameters['x'] = tile.x
    parameters['zoom'] = tile.z
    query_url = query_url.format(**parameters)
    return get_url(query_url, session, with_data=True, headers=extra_headers)


def get_wms_image(tile, source, session):
    bounds = list(mercantile.bounds(tile))
    if 'available_projections' not in source['properties']:
        return None
    available_projections = source['properties']['available_projections']
    url = source['properties']['url']
    proj = None
    if 'EPSG:3857' in available_projections:
        proj = 'EPSG:3857'
    elif 'EPSG:4326' in available_projections:
        proj = 'EPSG:4326'
    else:
        for proj in available_projections:
            try:
                CRS.from_string(proj)
            except:
                continue
            break
    if proj is None:
        return None

    crs_from = CRS.from_string("epsg:4326")
    crs_to = CRS.from_string(proj)
    if not proj == 'EPSG:4326':
        transformer = Transformer.from_crs(crs_from, crs_to, always_xy=True)
        bounds = list(transformer.transform(bounds[0], bounds[1])) + \
                 list(transformer.transform(bounds[2], bounds[3]))

    # WMS < 1.3.0 assumes x,y coordinate ordering.
    # WMS 1.3.0 expects coordinate ordering defined in CRS.
    if crs_to.axis_info[0].direction == 'north' and '=1.3.0' in url:
        bbox = ",".join(map(str, [bounds[1],
                                  bounds[0],
                                  bounds[3],
                                  bounds[2]]))
    else:
        bbox = ",".join(map(str, bounds))

    formatted_url = url.format(proj=proj,
                               width=256,
                               height=256,
                               bbox=bbox)

    return get_url(formatted_url, session, with_data=True)


def process_source(json_str, session):
    source = json.loads(json_str)
    
    if not source['properties']['type'] in {'tms', 'wms'}:
        print("Sources of type {} are currently not supported.".format(source['properties']['type']))

    if 'geometry' in source and source['geometry'] is not None:
        geom = shape(source['geometry'])
        centroid = geom.representative_point()
    else:
        centroid = Point(0, 0)


    def test_zoom(zoom):
        tile = mercantile.tile(centroid.x, centroid.y, zoom)

        if source['properties']['type'] == 'tms':
            response = get_tms_image(tile, source, session)
        elif source['properties']['type'] == 'wms':
            response = get_wms_image(tile, source, session)
            if response is None:
                return None, None, None

        if response is not None:
            img = Image.open(BytesIO(response))
            image_hash = imagehash.average_hash(img)
            pal_image = Image.new("P", (1, 1))
            pal_image.putpalette((0, 0, 0, 0, 255, 0, 255, 0, 0, 255, 255, 0) + (0, 0, 0) * 252)
            img_comp = img.convert("RGB").quantize(palette=pal_image)
            colors = img_comp.getcolors(1000)
            max_pixel_count = max([count for count, color in colors])
            return image_hash, img, max_pixel_count

        return None, None, None

    image_hashes = {}
    max_pixel_counts = {}
    images = {}
    for zoom in range(20):
        image_hash, img, max_pixel_count = test_zoom(zoom)
        images[zoom] = img
        image_hashes[zoom] = image_hash
        max_pixel_counts[zoom] = max_pixel_count

    # Getting images was not sucessful, nothing to do
    if len([zoom for zoom in range(20) if images[zoom] is None]) == len(range(20)):
        return

    def compare_neighbors(zoom):
        same_as_a_neighbor = False
        this_hash = image_hashes[zoom]
        if zoom - 1 >= 0:
            left_hash = image_hashes[zoom - 1]
            if left_hash == this_hash:
                same_as_a_neighbor = True
        if zoom + 1 < 20:
            right_hash = image_hashes[zoom + 1]
            if right_hash == this_hash:
                same_as_a_neighbor = True
        return same_as_a_neighbor

    def zoom_in_is_empty(zoom):
        if zoom + 1 < 20:
            if image_hashes[zoom + 1] is None or max_count(str(image_hashes[zoom + 1]).upper().replace('F', 'O')) == 16:
                return True
        return False

    # Find minzoom
    min_zoom = None
    for zoom in range(20):
        if image_hashes[zoom] is None:
            continue
        if zoom_in_is_empty(zoom):
            continue
        if max_count(str(image_hashes[zoom]).upper().replace('F', 'O')) == 16:
            continue
        if not compare_neighbors(zoom):
            min_zoom = zoom
            break

    fig, axs = plt.subplots(2, 10, figsize=(15, 5))
    for z in range(20):
        if z < 10:
            ax = axs[0][z]
        else:
            ax = axs[1][z - 10]

        ax.set_xlim(0, 256)
        ax.set_ylim(0, 256)
        if images[z] is not None:
            ax.imshow(images[z])
        else:
            ax.text(0.5, 0.5, 'No data', horizontalalignment='center',
                    verticalalignment='center', transform=ax.transAxes)

        ax.set_aspect('equal')
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        if image_hashes[z] is None:
            ax.set_xlabel("")
        else:
            ax.set_xlabel(str(image_hashes[z]) + "\n" + str(max_pixel_counts[z] - 256 * 256))
        #ax.set_ylabel(z)
        title = "Zoom: {}".format(z)

        if z == min_zoom:
            title += " <== "

        if ('min_zoom' not in source['properties'] and z == 0) or (
            'min_zoom' in source['properties'] and source['properties']['min_zoom'] == z):
            title += " ELI "

        ax.set_title(title)
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
print("Please copy paste geojson in the following input field and press enter.")
text = Text(
)
display(text)

session = requests.Session()

def handle_submit(sender):
    process_source(text.value, session)

text.on_submit(handle_submit)
